
# 🗺️ MelbSensors ⇢ Reverse Geocode por archivo (CSV a CSV)  
**Objetivo:** Tomar **cada** archivo (CSV o JSON) que tenga coordenadas y devolver **un CSV enriquecido por archivo**, conservando **todas** las columnas originales y agregando:  
`street`, `housenumber`, `postcode`, `city`, `country`, `formatted`.

### ✅ Qué hace este notebook
- Soporta **CSV** con `latitude/longitude` o `location=[lat, lon]` (y variantes) y **JSON** con `{"records":[{"fields":...}]}`.  
- Detecta separador del CSV automáticamente y muestra **diagnósticos** (columnas, filas, cómo detectó lat/lon).  
- Enriquecer **por archivo** → guarda `*_enriched.csv` con las nuevas columnas al final.  
- Incluye **cargador de archivos** de Colab y **montaje de Drive** (opcional).

> **Tip**: si tu CSV tiene otras cabeceras (p.ej. `Latitude`/`Longitude` con mayúsculas o `latitud`/`longitud` en español), este notebook las detecta. Si no, podés forzar el mapeo.



## 🔌 (Opcional) Montar Google Drive y/o subir archivos


In [ ]:

# Ejecuta esta celda si quieres usar Drive
try:
    from google.colab import drive  # type: ignore
    drive.mount('/content/drive')
except Exception:
    pass

# Subir archivos manualmente (si no los tienes en /content o Drive)
try:
    from google.colab import files  # type: ignore
    # Descomenta para abrir el selector de archivos
    # uploaded = files.upload()
except Exception:
    pass



## ⚙️ Configuración
- Pon tu **API key** de Geoapify (o define la variable de entorno `GEOAPIFY_API_KEY`).  
- Lista los archivos en `SOURCES`.  
- Si tu CSV usa encabezados no estándar, puedes **forzar** las columnas en `FORCE_COLUMN_MAP`.


In [ ]:

import os, json, math
from urllib.parse import urlparse

# API key (podés setearla por entorno o pegarla aquí)
GEOAPIFY_API_KEY = os.environ.get("GEOAPIFY_API_KEY", "").strip() or "c11f80da9db5447fbd62cee763865106"
assert GEOAPIFY_API_KEY, "Falta GEOAPIFY_API_KEY"

# Archivos/URLs a procesar: se crea un *_enriched.csv por cada uno
SOURCES = [
    # Ejemplos:
    # "/content/microclimate-sensor-locations.csv",
    # "/content/pedestrian-counting-system-sensor-locations.json",
]

# ⚠️ Solo si necesitás forzar nombres de columnas (por archivo)
# Claves: 'lat', 'lon' y opcionalmente 'location' (lista [lat, lon]).
# Usa el nombre EXACTO de la columna tal como aparece en tu CSV/JSON.
FORCE_COLUMN_MAP = {
    # "/content/microclimate-sensor-locations.csv": {"lat": "Latitude", "lon": "Longitude"},
    # "/content/pedestrian-counting-system-sensor-locations.csv": {"location": "Location"},
}

# Mínimo delay entre llamadas para ser amable con la API
REVERSE_SLEEP_SEC = 0.25



## 🧰 Utilidades (carga robusta, detección de columnas, reverse geocoding)


In [ ]:

import pandas as pd
import requests
from time import sleep

def _is_url(path: str) -> bool:
    try:
        return urlparse(path).scheme in {"http", "https"}
    except Exception:
        return False

def parse_json_records(data) -> pd.DataFrame:
    # JSON estilo {"records":[{"fields": {...}}]}
    if isinstance(data, dict) and "records" in data:
        rows = []
        for rec in data.get("records", []):
            fields = rec.get("fields", {})
            if isinstance(fields, dict):
                rows.append(fields)
        return pd.json_normalize(rows)
    # JSON lista de dicts
    if isinstance(data, list) and all(isinstance(x, dict) for x in data):
        return pd.json_normalize(data)
    return pd.DataFrame()

def read_csv_smart(path_or_url: str) -> pd.DataFrame:
    # Intentar con separador automático y diferentes encodings
    for sep in [None, ",", ";", "|", "	"]:
        for enc in ["utf-8", "utf-8-sig", "latin-1"]:
            try:
                return pd.read_csv(path_or_url, sep=sep, engine="python", encoding=enc)
            except Exception:
                continue
    # último intento por defecto
    return pd.read_csv(path_or_url)

def load_any(path_or_url: str) -> pd.DataFrame:
    if _is_url(path_or_url):
        # Intentar JSON
        try:
            r = requests.get(path_or_url, timeout=60)
            r.raise_for_status()
            try:
                return parse_json_records(r.json())
            except Exception:
                pass
        except Exception:
            pass
        # Intentar CSV
        return read_csv_smart(path_or_url)
    else:
        if not os.path.exists(path_or_url):
            raise FileNotFoundError(f"No existe: {path_or_url}")
        if path_or_url.lower().endswith(".json"):
            with open(path_or_url, "r", encoding="utf-8") as f:
                return parse_json_records(json.load(f))
        if path_or_url.lower().endswith(".csv"):
            return read_csv_smart(path_or_url)
        # Heurística
        try:
            with open(path_or_url, "r", encoding="utf-8") as f:
                return parse_json_records(json.load(f))
        except Exception:
            return read_csv_smart(path_or_url)

def extract_lat_lon_columns(df: pd.DataFrame, forced: dict | None = None):
    """
    Devuelve (lat, lon, used_from) como Series y etiqueta de origen ('latitude/longitude','location','forced','variants').
    Si no encuentra, devuelve (None, None, 'none').
    """
    if forced:
        # Forzar mapeo
        if "location" in forced and forced["location"] in df.columns:
            loc_col = forced["location"]
            lat, lon = _latlon_from_location_series(df[loc_col])
            return lat, lon, "forced(location)"
        if "lat" in forced and "lon" in forced and forced["lat"] in df.columns and forced["lon"] in df.columns:
            lat = pd.to_numeric(df[forced["lat"]], errors="coerce")
            lon = pd.to_numeric(df[forced["lon"]], errors="coerce")
            return lat, lon, "forced(lat/lon)"

    lower = {c.lower(): c for c in df.columns}

    # Caso directo: latitude/longitude (insensible a mayúsculas)
    if "latitude" in lower and "longitude" in lower:
        lat = pd.to_numeric(df[lower["latitude"]], errors="coerce")
        lon = pd.to_numeric(df[lower["longitude"]], errors="coerce")
        return lat, lon, "latitude/longitude"

    # Desde 'location' (lista [lat, lon] o [lon, lat])
    if "location" in lower:
        lat, lon = _latlon_from_location_series(df[lower["location"]])
        return lat, lon, "location"

    # Variantes comunes
    lat_col = None
    for a in ["lat", "latitude_deg", "Lat", "Latitude"]:
        if a in df.columns:
            lat_col = a
            break
        if a.lower() in lower:
            lat_col = lower[a.lower()]
            break
    lon_col = None
    for b in ["lon", "lng", "longitude_deg", "Long", "Longitude", "Lng"]:
        if b in df.columns:
            lon_col = b
            break
        if b.lower() in lower:
            lon_col = lower[b.lower()]
            break

    if lat_col and lon_col:
        lat = pd.to_numeric(df[lat_col], errors="coerce")
        lon = pd.to_numeric(df[lon_col], errors="coerce")
        return lat, lon, "variants"

    return None, None, "none"

def _latlon_from_location_series(series: pd.Series):
    def _extract(x):
        if isinstance(x, (list, tuple)) and len(x) >= 2:
            a, b = x[0], x[1]
            if isinstance(a, (int, float)) and isinstance(b, (int, float)):
                if -90 <= a <= 90 and -180 <= b <= 180:
                    return a, b
                if -90 <= b <= 90 and -180 <= a <= 180:
                    return b, a
        # Si llega como string tipo "[lat, lon]"
        if isinstance(x, str) and "[" in x and "]" in x:
            try:
                vals = json.loads(x)
                if isinstance(vals, list) and len(vals) >= 2:
                    a, b = vals[0], vals[1]
                    if -90 <= a <= 90 and -180 <= b <= 180:
                        return a, b
                    if -90 <= b <= 90 and -180 <= a <= 180:
                        return b, a
            except Exception:
                pass
        return (math.nan, math.nan)
    ll = series.apply(_extract)
    lat = pd.Series([v[0] for v in ll], index=series.index)
    lon = pd.Series([v[1] for v in ll], index=series.index)
    return lat, lon

def reverse_geocode(lat: float, lon: float, api_key: str, session: requests.Session, max_retries: int = 3):
    url = "https://api.geoapify.com/v1/geocode/reverse"
    params = {"lat": float(lat), "lon": float(lon), "format": "json", "apiKey": api_key}
    for attempt in range(1, max_retries + 1):
        try:
            r = session.get(url, params=params, timeout=30)
            if r.status_code == 200:
                d = r.json()
                if isinstance(d, dict) and d.get("results"):
                    return d["results"][0]
                return None
            if r.status_code in (429, 500, 503):
                sleep(1.5 * attempt)  # backoff simple
            else:
                return None
        except requests.RequestException:
            sleep(1.5 * attempt)
    return None

def addr_fields(res: dict) -> dict:
    return {
        "street": res.get("street") if res else None,
        "housenumber": res.get("housenumber") if res else None,
        "postcode": res.get("postcode") if res else None,
        "city": (res.get("city") or res.get("district") or res.get("county")) if res else None,
        "country": res.get("country") if res else None,
        "formatted": res.get("formatted") if res else None,
    }

def stem_for_output(path_or_url: str) -> str:
    if _is_url(path_or_url):
        parsed = urlparse(path_or_url)
        name = os.path.basename(parsed.path) or "dataset"
    else:
        name = os.path.basename(path_or_url)
    stem = os.path.splitext(name or "dataset")[0] or "dataset"
    return stem



## 🔍 Diagnóstico de archivos
Muestra: filas, columnas, primeras filas y cómo detectó las coordenadas.  
Si ves `source: none`, ajusta `FORCE_COLUMN_MAP` para ese archivo.


In [ ]:

from tqdm.auto import tqdm

for src in SOURCES:
    print(f"\n==> Analizando: {src}")
    try:
        df = load_any(src)
    except Exception as e:
        print(f"   ❌ Error al cargar: {e}")
        continue

    if df.empty:
        print("   ⚠️ DataFrame vacío.")
        continue

    print(f"   ✔️ Shape: {df.shape[0]} filas x {df.shape[1]} columnas")
    print("   ✔️ Columnas:", list(df.columns)[:50], ("..." if len(df.columns) > 50 else ""))
    display(df.head(3))

    forced = FORCE_COLUMN_MAP.get(src)
    lat, lon, used_from = extract_lat_lon_columns(df, forced=forced)
    print(f"   🔎 Detección de coordenadas: {used_from}")
    if lat is not None and lon is not None:
        valid = int((lat.notna() & lon.notna()).sum())
        print(f"   ✔️ Coordenadas válidas: {valid} filas")
    else:
        print("   ⚠️ No se detectaron columnas de coordenadas. Considera FORCE_COLUMN_MAP.")



## ▶️ Enriquecer y exportar (CSV por archivo)
- Conservar todas las columnas originales; añade las nuevas al final.  
- Si alguna fila no tiene coordenadas válidas, deja columnas nuevas vacías.


In [ ]:

session = requests.Session()
cache = {}
created = []

for src in SOURCES:
    print(f"\n==> Procesando: {src}")
    try:
        df = load_any(src)
    except Exception as e:
        print(f"   ❌ Error al cargar: {e}")
        continue
    if df.empty:
        print("   ⚠️ DataFrame vacío, se omite.")
        continue

    forced = FORCE_COLUMN_MAP.get(src)
    lat, lon, used_from = extract_lat_lon_columns(df, forced=forced)
    if lat is None or lon is None:
        print("   ⚠️ No hay columnas de coordenadas detectadas; se guardará sin enriquecimiento.")
    else:
        print(f"   🔎 Usando columnas de coordenadas desde: {used_from}")

    # Copia exacta; columnas nuevas al final
    out = df.copy()
    for c in ["street", "housenumber", "postcode", "city", "country", "formatted"]:
        if c not in out.columns:
            out[c] = None

    if lat is not None and lon is not None:
        for idx in tqdm(out.index, desc=f"Reverse geocoding ({stem_for_output(src)})"):
            la = lat[idx] if isinstance(lat, pd.Series) and idx in lat.index else math.nan
            lo = lon[idx] if isinstance(lon, pd.Series) and idx in lon.index else math.nan
            if pd.notna(la) and pd.notna(lo):
                key = (round(float(la), 6), round(float(lo), 6))
                if key not in cache:
                    res = reverse_geocode(float(la), float(lo), GEOAPIFY_API_KEY, session=session)
                    cache[key] = addr_fields(res)
                    sleep(REVERSE_SLEEP_SEC)
                for k, v in cache[key].items():
                    out.at[idx, k] = v

    out_name = f"{stem_for_output(src)}_enriched.csv"
    out.to_csv(out_name, index=False, encoding="utf-8")
    created.append(out_name)
    print(f"   ✅ Guardado: {out_name}")

print("\nArchivos generados:")
for c in created:
    print(" -", c)



### 📝 Tips si te dice “no hay nada”
- Verifica la **ruta** exacta (`/content/...`) o que el archivo esté **subido** (usa el selector de archivos arriba).  
- Revisa la celda de **Diagnóstico**: mira las columnas y confirma que existen `latitude/longitude` o `location`.  
- Si tus columnas se llaman distinto, usa `FORCE_COLUMN_MAP` en Configuración:
```python
FORCE_COLUMN_MAP = {
  "/content/microclimate-sensor-locations.csv": {"lat": "Latitude", "lon": "Longitude"},
  # O bien si viene en una lista:
  # "/content/pedestrian.csv": {"location": "Location"},
}
```
- CSV con separador `;` o con BOM: el lector automático lo detecta; si ves columnas raras, abre la **Vista previa** en Diagnóstico para confirmar.
- Si sale error 429, sube `REVERSE_SLEEP_SEC` a `0.4` o más.
